In [ ]:
from tensorflow.keras import *
from tensorflow.keras.layers import *
from generations import *
import tensorflow as tf
import numpy as np
import keras_tuner as kt
import matplotlib.pyplot as plt

In [ ]:
data_test = np.load('channel_test.npy')
data_train = np.load('channel_train.npy')
data_test.shape


In [ ]:
def bit_err(y_true, y_pred):
    err = 1 - tf.reduce_mean(
        tf.reduce_mean(
            #tf.to_float(
            float(
                tf.equal(
                    tf.sign(
                        y_pred - 0.5),
                    tf.cast(
                        tf.sign(
                            y_true - 0.5),
                        tf.float32))),
            1))
    return err

In [ ]:
input_bits = Input(shape=(payloadBits_per_OFDM * 2,))

In [ ]:
temp = BatchNormalization()(input_bits)
temp = Dense(n_hidden_1, activation='relu')(input_bits)
temp = BatchNormalization()(temp)
temp = Dense(n_hidden_2, activation='relu')(temp)
temp = BatchNormalization()(temp)
temp = Dense(n_hidden_3, activation='relu')(temp)
temp = BatchNormalization()(temp)
out_put = Dense(n_output, activation='sigmoid')(temp)
model = Model(input_bits, out_put)
model.compile(optimizer='adam', loss='mse', metrics=[bit_err])
model.summary()
checkpoint = callbacks.ModelCheckpoint('./temp_trained_25.h5', monitor='val_bit_err',
                                       verbose=0, save_best_only=True, mode='min', save_weights_only=True)

In [ ]:
model.fit(
    training_gen(500,25), #(1000,25),
    steps_per_epoch=20, #50,
    epochs=2000, #10000
    validation_data=validation_gen(500,25), #(1000,25),
    validation_steps=1,
    callbacks=[checkpoint],
    verbose=2)

In [ ]:
model.load_weights('./temp_trained_25.h5')

In [ ]:
BER = []
for SNR in range(5, 30, 5):
    y = model.evaluate(
        validation_gen(500, SNR),
        steps=1
    )
    BER.append(y[1])
    print(y)
print(BER)
BER_matlab = np.array(BER)
#import scipy.io as sio
#sio.savemat('BER.mat', {'BER':BER_matlab})

SNR_arr = np.linspace(5,25,5)
plt.plot(SNR_arr,BER_matlab)
plt.ylabel('BER')
plt.xlabel('SNR (dB)')
plt.yscale('log')
plt.ylim(1e-3,1e0)
plt.title('BER vs SNR for OFDM System with DNN Receiver')
plt.grid('on')